# Prototype Curves – Week 1

**Goals**
1. Load `docs/literature_matrix.csv`.
2. Plot F1 vs. annotator minutes for *Dragut et al., 2019*.
3. Plot accuracy vs. dollars for *Kang et al., 2023*.
4. Illustrate diminishing returns & marginal-gain stop point.

_Code cells will be added once I’m on a machine with Jupyter._


In [ ]:
# === Load CSV ===
import pandas as pd
df = pd.read_csv('../docs/literature_matrix.csv')
df.head()

In [ ]:
# === Dragut curve ===
from src.utils import parse_curve
import matplotlib.pyplot as plt
row = df[df['Paper'].str.contains('Dragut')].iloc[0]
x, y = zip(*parse_curve(row['Key Curve Data']))
plt.plot(x, y, marker='o')
plt.xlabel('Annotator minutes')
plt.ylabel('F1')
plt.title('Dragut 2019 – Diminishing returns')
plt.show()